In [ ]:
%cd bento

/home/jaemin/llm-cxr/bento


In [2]:
from typing import TYPE_CHECKING, cast
import PIL
import bentoml
from pydantic import BaseModel
from bentoml.io import JSON, Multipart, Text, NumpyNdarray, Image
import numpy as np
import os
import torch
#import torchvision.transforms.functional as TF
#import torchvision.transforms as T
#import PIL

#if TYPE_CHECKING:
    #from PIL.Image import Image


TARGET_IMAGE_SIZE = 256

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # for debugging purpose
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # for debugging purpose

rrg_runner = bentoml.models.get("brain-mri-rrg:latest").to_runner()

svc = bentoml.Service(name="brain-mri-rrg", runners=[rrg_runner])


class TextResponseDTO(BaseModel):
    answer: str

    class Config:
        extra = "forbid"
        arbitrary_types_allowed = True

# ImageToText
@svc.api(
    input=Multipart(query=Text(), img=NumpyNdarray()),
    output=JSON(pydantic_model=TextResponseDTO),
)

async def reportgen(query: str, img: np.ndarray) -> TextResponseDTO:

    c = img.size[3]
    m = max(img.size)
    
    if c != 3:
        raise ValueError(f"Incorrect number of channels for MRI image processing")

    if m < TARGET_IMAGE_SIZE:
        raise ValueError(f"min dim for image {s} < {TARGET_IMAGE_SIZE}")
        
    ###Prompt Generation
    img_tag = '<Img><ImageHere></Img>'
    prompt = img_tag + query
    model_input = (img, prompt)

    ###Report Generation
    answer = await rrg_runner.async_run(model_input, max_new_tokens=200)
    return TextResponseDTO(
        answer=answer,
    )



/home/jaemin/.pyenv/versions/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
for runner in svc.runners:
    runner.init_local()

'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.
'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.
'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.


In [ ]:
inp = {
    "query": "Describe this volumetric brain tumor MRI with possible abnormalities.",
    "img": np.load(''), 
}
result = await svc.apis["reportgen"].func(**inp)

In [5]:
print(result)

TextResponseDTO(answer='Right upper lobe pneumonia.  Recommend followup to resolution.')